In [21]:
from selenium import webdriver 
import time
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
driver = webdriver.Chrome(options=chrome_options)
start_url = "https://www.bajajfinservmarkets.in/discover/journals/blogs/page/1/"
driver.get(start_url)
time.sleep(5)
# print(driver.page_source.encode("utf-8"))
a = driver.find_elements_by_class_name("page-link")
pages = int(a[-2].get_attribute('innerHTML'))
# print(a)
print(pages)

262


In [55]:
links = []

for page in range(1, pages+1):
    path = "https://www.bajajfinservmarkets.in/discover/journals/blogs/page/" + str(page) + "/"
    driver.get(path)
    time.sleep(0.5)
    section = driver.find_elements_by_class_name("articles-listing")[0]
#     print(section)
    posts = section.find_elements_by_css_selector('div.row')
#     print(posts)
    for postDiv in posts:
        post = postDiv.find_elements_by_css_selector('div.brand-white-bg')[-1]
        linkOBJ = post.find_elements_by_css_selector('a')
        link = ""
        for x in linkOBJ:
            if (x.get_attribute('innerHTML') == "READ MORE"):
                link = x.get_attribute('href')
#         print(link)
        links.append(link)
    time.sleep(0.5)

    

KeyboardInterrupt: 

In [56]:
# for blog in links:
print(links)

['https://www.bajajfinservmarkets.in/discover/journals/blogs/investment/5-personal-finance-tips-that-will-help-you-manage-your-money-better', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/investment/cryptocurrencies-digital-gold-or-a-ponzi-scheme', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/service/benefits-of-advance-emi-payment', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/insurance/how-are-men-and-women-different-in-retirement-planning', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/insurance/why-does-india-spend-so-much-on-out-of-pocket-medical-expenses', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/loans/top-4-benefits-of-a-personal-loan', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/insurance/introduction-to-basic-yoga-sequences', 'https://www.bajajfinservmarkets.in/discover/journals/blogs/emi-card/end-of-season-sale-with-bajaj-finserv-emi-card', 'https://www.bajajfinservmarkets.in/discover

In [70]:
!pip install contractions
!pip install fuzzywuzzy
!pip install TextBlob
!pip install transformers

  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp38-cp38-win_amd64.whl size=38173 sha256=0c3aa71674af7fb3749df73e8815396e14daba4caad91030b98d2bbd5fea79f3
  Stored in directory: c:\users\anmol\appdata\local\pip\cache\wheels\74\bc\b8\e5f739a84005620cfe66d3fcb8bb182e309d6056bc6700b60e
Successfully built pyahocorasick
^C
^C
^C


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.0.12 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


In [71]:


from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import contractions
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')
from textblob import TextBlob
from fuzzywuzzy import fuzz
from transformers import pipeline,BertTokenizer, TFBertModel
import joblib
import tensorflow as tf
from scipy.spatial import distance  

"""
# to be run only once
nlp_qa = pipeline('question-answering')
summarizer = pipeline('summarization')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertModel.from_pretrained("bert-base-uncased")
joblib.dump(summarizer,"/content/drive/MyDrive/Hackathon/summarizer.joblib")
joblib.dump(nlp_qa,"/content/drive/MyDrive/Hackathon/question_answering.joblib")
model.save_pretrained("/content/drive/MyDrive/Hackathon/Bert_model",save_format="tf")
joblib.dump(tokenizer,"/content/drive/MyDrive/Hackathon/Bert_tokenizer.joblib")
"""


class QueryProcessor:
    def __init__(self):
        try:
            self.summarizer = joblib.load("process/Models/summarizer.joblib")
            self.nlp_qa = joblib.load("process/Models/question_answering.joblib")
            self.model = TFBertModel.from_pretrained("process/Models/Bert_model")
            self.tokenizer = joblib.load("process/Models/Bert_tokenizer.joblib")
        except Exception as e:
            print(e)
        finally:
            self.lem = WordNetLemmatizer()

    def generic_preprocessor(self, text):
        text = str(text)
        text = text.encode('ascii', 'ignore')
        text = text.decode()
        text = text.lower()
        text = str(TextBlob(text).correct())
        text = contractions.fix(text)
        text = ' '.join([word for word in text.split()
                         if not word in set(stopwords.words('english'))])
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = ' '.join(text.split())
        text = self.lem.lemmatize(text)
        return text

    def fuzz_matching(self, quest1, quest2):
        if type(quest1) == list:
            quest1 = " ".join(quest1)
        if type(quest2) == list:
            quest2 = " ".join(quest2)
        quest1 = generic_preprocessor(quest1)
        quest2 = generic_preprocessor(quest2)
        return fuzz.partial_ratio(quest1, quest2)

    def keyword_extraction(self, text):
        return list(TextBlob(text).noun_phrases)

    def question_answering(self, content, question, rec=False):
        if rec == False and len(content.split(" ")) < 10:
            return " "
        elif rec == True and len(content) < 10:
            return (0, " ")

        elif rec == False and 10 < len(content.split(" ")) < 500:
            return self.nlp_qa(context=content, question=question)["answer"]

        elif rec == True and 10 < len(content) < 500:
            content = " ".join(content)
            answer = self.nlp_qa(context=content, question=question)
            return (round(answer["score"], 3), answer["answer"])

        else:
            content = content.split(" ")
            # split on basis of senetnce
            content = [content[i:i+499] for i in range(0, len(content), 499)]
            content = [question_answering(
                i, question, rec=True) for i in content]
            possible_answers = sorted(
                content, key=lambda x: x[0], reverse=True)
            if len(possible_answers) == 1:
                return possible_answers[0][1]
            else:
                if possible_answers[0][0] == possible_answers[1][0]:
                    return possible_answers[0][1]
                elif 0 < ((possible_answers[0][0] - possible_answers[1][0])/possible_answers[1][0]) < 0.05:
                    return possible_answers[0][1] + " | " + possible_answers[1][1]
                else:
                    return possible_answers[0][1]

    def summarization(self, text):
        if type(text) == str:
            if len(text.split(" ")) < 10:
                return text
            if len(text.split(" ")) < 500:
                return self.summarizer(text)[0]["summary_text"]
            else:
                text = text.split(" ")
                # split on basis of sentence
                text = [text[i:499] for i in range(0, len(text), 499)]
                text = [summarization(i) for i in text]
                # can be improved with fuzzy match from list of summarized don't chose text with higher similarity to other
                return " ".join(text)
        else:
            if len(text) < 10:
                return " ".join(text)
            text = " ".join(text)
            return self.summarizer(text)[0]["summary_text"]

    def semantic_search(self,text1, text2):
        inputs1 = self.tokenizer(
            [text1], add_special_tokens=True, return_tensors="tf")
        inputs2 = self.tokenizer(
            [text2], add_special_tokens=True, return_tensors="tf")
        output1 = self.model(inputs1)[-1].numpy()
        output2 = self.model(inputs2)[-1].numpy()
        return (1 - distance.cosine(output1, output2))*100

"""
Q = QueryProcessor()
Q.generic_preprocessor("Hello, how are yuo?")
"""

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anmol\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anmol\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\anmol\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anmol\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
D:\Programming\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/

ModuleNotFoundError: No module named 'tensorflow'

In [68]:
blogs = []
l = ['investment', 'loans', 'emi-card']
for link in links:
    category = link.split('/')[-2]
#     print(category)
    if (category in l):        
        driver.get(link)
        time.sleep(0.25)
        title = driver.find_elements_by_css_selector('h1.brand-primary')[0]
        body = driver.find_elements_by_css_selector('div.post-content-wrapper')[0]
#         print(title.get_attribute('innerText'))
#         print(body.get_attribute('innerHTML'))
        blogs.append({
            "title": title.get_attribute('innerText'),
            "content": body.get_attribute('innerHTML'),
            "category": category,
            "link": link
        })
#         break

In [69]:
for blog in blogs:
    print(blog)
    print()

{'title': '5 Personal Finance Tips That Will Help You Manage Your Money Better', 'content': '\n  \n  <p style="text-align: justify;">Cracking the code to personal finance may not seem easy at first. But if you have the right tips guiding you, you’ll find that managing your money becomes so much more effortless. Mastering personal finance can help you answer all those questions you may have had over the years, like how you can go about saving money and making the right investment decisions.</p>\n<p style="text-align: justify;">So, let’s take a look at 5 personal finance tips that will help you get better at personal finance.</p>\n<ul style="text-align: justify;">\n<li>\n<h2><b>Budget. Budget. Budget.</b></h2>\n</li>\n</ul>\n<p style="text-align: justify;">Budgeting is the first step to getting better at personal finance. Creating a budget is crucial because it gives you a complete idea of the money that is coming in, and the money that is being spent. This information is fundamental to 

In [ ]:
driver.quit()